## 1. First, make some data preprocessing to clean up the data. You can use your solution to the assignment of the data preprocessing checkpoint of this module.

In [5]:
pip install chatterbot

  Using cached ChatterBot-1.0.5-py2.py3-none-any.whl (67 kB)
Processing c:\users\jonat\appdata\local\pip\cache\wheels\26\f5\25\6f2227c446ba93eef8dc8544d76a51a67a10c66066ec216d5d\sqlalchemy-1.2.19-cp37-cp37m-win_amd64.whl
  Using cached PyYAML-5.1.2-cp37-cp37m-win_amd64.whl (215 kB)
  Using cached python_dateutil-2.7.5-py2.py3-none-any.whl (225 kB)
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.3.17
    Uninstalling SQLAlchemy-1.3.17:
      Successfully uninstalled SQLAlchemy-1.3.17
Note: you may need to restart the kernel to use updated packages.


ERROR: fbprophet 0.6 has requirement python-dateutil>=2.8.0, but you'll have python-dateutil 2.7.5 which is incompatible.
ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'c:\\users\\jonat\\anaconda3\\lib\\site-packages\\~~lalchemy\\cprocessors.cp37-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [6]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import nltk
import spacy
import re
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer, ChatterBotCorpusTrainer
from chatterbot.conversation import Statement
import random

import warnings
warnings.filterwarnings(action="ignore")

!python -m spacy download en

ModuleNotFoundError: No module named 'chatterbot'

In [4]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'cornell_movie_dialogs'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

dialogs_df = pd.read_sql_query('select * from dialogs',con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()

In [5]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
nlp.add_pipe(nlp.create_pipe('sentencizer'))

# below is necessary to avoid memory error of SpaCy
nlp.max_length = 20000000

# all the processing work is done below, so it may take a while
dialogs_doc = nlp(" ".join(dialogs_df.dialogs))

In [ ]:
dialog_sents = [sent.text for sent in dialogs_doc.sents if len(sent.text) > 1]

## 2. Develop a chatbot using this corpus. In doing this, you're free to choose a chatbot development libary like ChatterBot or to write your own code from scratch.

In [ ]:
GREETING_INPUTS = ["hello", "hi", "greetings", "what's up","hey"]
GREETING_RESPONSES = ["hello", "hi", "hey", "hi there"]
def greeting(sentence):
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

In [ ]:
# create a chatbot
chatbot = ChatBot('Dialogs')
# this is to remove the accumulated knowledge base
chatbot.storage.drop()

# create a new trainer for the chatbot
trainer = ListTrainer(chatbot)

# train the chatbot
trainer.train(dialog_sents)

## 3. Make a conversation with your chatbot and discuss the strengths and weaknesses of it.

In [ ]:
import random

In [ ]:
print("Cornell: I will try to respond you reasonably. If you want to exit, type bye please.")

# below is the chatting
while True:
    
    user_input = input("User: ")
    user_input=user_input.lower()
    
    if(user_input!='bye'):
        if(user_input == 'thanks' or user_input == 'thank you'):
            break
            print("Cornell: You're welcome.")
        else:
            if(greeting(user_input) != None):
                print("Cornell: " + greeting(user_input))
            else:
                print("Cornell: ", end = "")
                print(chatbot.get_response(user_input))
    else:
        print("Cornell: Bye! It was a great chat.")
        break

Not bad, right? We can really hear the Hollywood jargon in the conversation. As expected, our bot carries the colors and the biases in the movies!